In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame(pd.read_csv("../assets/normalized_pyramid_data.csv"))

## Camilo's sample queries (6/26)

In [ ]:
'''
df[df["dynasty"] == 3][df["exterior_material"] == "Limestone"][df["start_of_reign"] <= 2700][df["end_of_reign"] >= 2550]

df[df["superstructure_type"] == "Pyramid"]["width"]

len(df[df["gender"] == "King"][df["period"] == "Old Kingdom"])

df[df["dynasty"] >= 5][df["dynasty"] <= 7][df["state_of_completion"] == "Completed"]
'''

In [ ]:
kings = df[df['gender'] == 'King']

In [ ]:
# Not sure of the point of this; Brought over from visualizations

start_reign = df["start_of_reign"].notnull()
end_reign = df["end_of_reign"].notnull()
reign_df = df[start_reign & end_reign]

In [ ]:
# Find the number of queen pyramids at each complex

queens = df[df['gender'] == 'Queen']
queens[['gender', 'pyramid_complex']].groupby('pyramid_complex').count()

In [ ]:
# Compare the number of king and queen pyramids at each confirmed complex

key = ['unknown', 'pyramid?']
complexes = df[~df['pyramid_complex'].isin(key)]

kings_and_queens = complexes.groupby('pyramid_complex')['gender'].value_counts().unstack()
kings_and_queens

In [ ]:
plt = kings_and_queens.plot.bar(xlabel='Pyramid Complex', ylabel='Number of Confirmed Pyramids')
plt.legend(title='Royal Status')

In [ ]:
# Count all known casing types

df[df['casing'] != 'Unknown']['casing'].value_counts()

In [ ]:
# See the types of casing during each dyansty

dynasty_casing = df.groupby('dynasty')['casing'].value_counts().unstack()
dynasty_casing

In [ ]:
# See the distribution of pyramids between genders among the dynasties

dynasty_pyramids = df.groupby('dynasty')['gender'].value_counts().unstack()
dynasty_pyramids

In [ ]:
plt = dynasty_pyramids.plot.bar(xlabel='Dyansty', ylabel='Number of Pyramids (Including Speculations)')
plt.legend(title='Royal Status')

In [ ]:
queen_data = queens[['royal_mother_title', 'likely_wife', 'wife_title', 'vizier', 'regent']]
queen_data

In [ ]:
queen_data.value_counts()